# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f08c95329d0>
├── 'a' --> tensor([[ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128]])
└── 'x' --> <FastTreeValue 0x7f08c9532c70>
    └── 'c' --> tensor([[ 1.7312, -0.3089,  0.3019,  1.9638],
                        [-0.0581,  0.1818, -0.6221, -0.2366],
                        [ 1.6376, -0.4515,  0.6696,  0.2338]])

In [4]:
t.a

tensor([[ 1.3091, -0.3315,  0.3347],
        [-0.2215,  0.3643,  1.3128]])

In [5]:
%timeit t.a

66.8 ns ± 0.0515 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f08c95329d0>
├── 'a' --> tensor([[-0.5264, -0.4553, -0.1173],
│                   [ 0.8574,  0.1487, -1.0452]])
└── 'x' --> <FastTreeValue 0x7f08c9532c70>
    └── 'c' --> tensor([[ 1.7312, -0.3089,  0.3019,  1.9638],
                        [-0.0581,  0.1818, -0.6221, -0.2366],
                        [ 1.6376, -0.4515,  0.6696,  0.2338]])

In [7]:
%timeit t.a = new_value

72 ns ± 0.153 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128]]),
    x: Batch(
           c: tensor([[ 1.7312, -0.3089,  0.3019,  1.9638],
                      [-0.0581,  0.1818, -0.6221, -0.2366],
                      [ 1.6376, -0.4515,  0.6696,  0.2338]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3091, -0.3315,  0.3347],
        [-0.2215,  0.3643,  1.3128]])

In [11]:
%timeit b.a

59 ns ± 0.0497 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6563,  0.3406,  0.1921],
               [ 1.1412, -0.4664,  0.1829]]),
    x: Batch(
           c: tensor([[ 1.7312, -0.3089,  0.3019,  1.9638],
                      [-0.0581,  0.1818, -0.6221, -0.2366],
                      [ 1.6376, -0.4515,  0.6696,  0.2338]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

824 ns ± 0.716 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 21.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 572 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0820dd08b0>
├── 'a' --> tensor([[[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]],
│           
│                   [[ 1.3091, -0.3315,  0.3347],
│                    [-0.2215,  0.3643,  1.3128]]])
└── 'x' --> <FastTreeValue 0x7f0820dd0670>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 63 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f08c9532eb0>
├── 'a' --> tensor([[ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128],
│                   [ 1.3091, -0.3315,  0.3347],
│                   [-0.2215,  0.3643,  1.3128]])
└── 'x' --> <FastTreeValue 0x7f081a13eac0>
    └── 'c' --> tensor([[ 1.7312, -0.3089,  0.3019,  1.9638],
                        [-0.0581,  0.1818, -0.6221, -0.2366],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 91.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]],
       
               [[ 1.3091, -0.3315,  0.3347],
                [-0.2215,  0.3643,  1.3128]]]),
    x: Batch(
           c: tensor([[[ 1.7312, -0.3089,  0.3019,  1.9638],
                       [-0.0581,  0.1818, -0.6221, -0.2366],
                       [ 1.6376, -0.4515,  0.6696,  0.2338]],
         

In [26]:
%timeit Batch.stack(batches)

140 µs ± 371 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128],
               [ 1.3091, -0.3315,  0.3347],
               [-0.2215,  0.3643,  1.3128]]),
    x: Batch(
           c: tensor([[ 1.7312, -0.3089,  0.3019,  1.9638],
                      [-0.0581,  0.1818, -0.6221, -0.2366],
                      [ 1.6376, -0.4515,  0.6696,  0.2338],
                      [ 1.7312, -0.3089,  0.3019,  1.9638],
                      [-0.0581,  

In [28]:
%timeit Batch.cat(batches)

454 µs ± 7.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
